#### 4-10. 분산의 중요성
* 평균값은 동일하지만 분산값이 다름으로 인해 전체적인 데이터의 모습이 완전히 달라 보임
* 분산이 클수록 집단의 평균값의 차이가 무의미해진다.
* **집단간 평균값의 분산이 클수록, 집단 내 분산이 작아질 수록 평균의 차이가 분명해진다.**

#### 4-02. 일원분산분석(One-way ANOVA)
* 독립변수: 범주형 1개, 종속변수: 연속형
* 독립변수의 변화가 종속변수에 미치는 영향을 보기 위해 사용
* 가정: 독립성, 정규성, 등분산성
* scipy.stats나 statsmodels 라이브러리 사용

* **iris 데이터에서, 품종(species)별 sepal_length의 평균에 차이가 있을까?**
* 독립변수: species
* 종속변수: sepal_length, sepal_width, petal_length, petal_width 중 1개
* One-way ANOVA를 통해 평균 차이에 대한 통계적 유의성을 알아 볼 수 있다.
* **귀무가설: 세 품종의 sepal_width 평균은 동일하다.**
* **대립가설: 적어도 한 품종의 sepal_width 평균이 다르다.**

In [4]:
# [1] 데이터 가져오기
import pandas as pd
import seaborn as sns

iris = sns.load_dataset('iris')
print(iris.head(3))

   sepal_length  sepal_width  petal_length  petal_width species
0           5.1          3.5           1.4          0.2  setosa
1           4.9          3.0           1.4          0.2  setosa
2           4.7          3.2           1.3          0.2  setosa


In [8]:
# [2] 그룹나누기
target = 'sepal_width'
# print(iris['species'].unique())
gA, gB, gC = [s.to_numpy() for _, s in iris.groupby('species')[target]]
print(gA, gB, gC, sep="\n")

[3.5 3.  3.2 3.1 3.6 3.9 3.4 3.4 2.9 3.1 3.7 3.4 3.  3.  4.  4.4 3.9 3.5
 3.8 3.8 3.4 3.7 3.6 3.3 3.4 3.  3.4 3.5 3.4 3.2 3.1 3.4 4.1 4.2 3.1 3.2
 3.5 3.6 3.  3.4 3.5 2.3 3.2 3.5 3.8 3.  3.8 3.2 3.7 3.3]
[3.2 3.2 3.1 2.3 2.8 2.8 3.3 2.4 2.9 2.7 2.  3.  2.2 2.9 2.9 3.1 3.  2.7
 2.2 2.5 3.2 2.8 2.5 2.8 2.9 3.  2.8 3.  2.9 2.6 2.4 2.4 2.7 2.7 3.  3.4
 3.1 2.3 3.  2.5 2.6 3.  2.6 2.3 2.7 3.  2.9 2.9 2.5 2.8]
[3.3 2.7 3.  2.9 3.  3.  2.5 2.9 2.5 3.6 3.2 2.7 3.  2.5 2.8 3.2 3.  3.8
 2.6 2.2 3.2 2.8 2.8 2.7 3.3 3.2 2.8 3.  2.8 3.  2.8 3.8 2.8 2.8 2.6 3.
 3.4 3.1 3.  3.1 3.1 3.1 2.7 3.2 3.3 3.  2.5 3.  3.4 3. ]


In [9]:
# [3] 품종별 각 변수의 평균 확인
print(gA.mean(), round(gB.mean(), 3), gC.mean())

3.428 2.77 2.974


In [10]:
# [4] 일원분산분석
# 1) model = OLS.from_formula()
# 2) anova_lm(model)
from statsmodels.api import OLS
from statsmodels.stats.anova import anova_lm

formula = 'sepal_width ~ C(species)'
model = OLS.from_formula(formula, data=iris).fit()
result = anova_lm(model)
print(result)

               df     sum_sq   mean_sq         F        PR(>F)
C(species)    2.0  11.344933  5.672467  49.16004  4.492017e-17
Residual    147.0  16.962000  0.115388       NaN           NaN


In [19]:
# [5] 오차제곱합, SSE(Sum of Squares Error)
SSE = result.loc['Residual', 'sum_sq']
print(round(result, 6))

# [6] 회귀제곱합, SSR(Sum of Squares of Regression)
SSR = result.loc['C(species)', 'sum_sq']
print(round(SSR, 6))

# [7] 결정계수
R2 = SSR / (SSR + SSE)
# R2 = SSR / result['sum_sq'].mean()
print(round(R2, 3))

# [8] 평균제곱오차(MSE, Mean Squared Error)
MSE = result.loc['Residual', 'mean_sq']
print(round(MSE, 6))

# [9] 평균제곱회귀(MSR, Mena Squared Regression)
MSR = result.loc['C(species)', 'mean_sq']
print(round(MSR, 6))

# [10] F-통계량
F = result.loc['C(species)', 'F']
print(round(F, 6))

# [11] F-통계량 - pvalue
pvalue = result.loc['C(species)', 'PR(>F)']
print(round(pvalue, 6))

               df     sum_sq   mean_sq         F  PR(>F)
C(species)    2.0  11.344933  5.672467  49.16004     0.0
Residual    147.0  16.962000  0.115388       NaN     NaN
11.344933
0.401
0.115388
5.672467
49.16004
0.0


In [20]:
# [5] 사후검정
from statsmodels.stats.multicomp import pairwise_tukeyhsd
result = pairwise_tukeyhsd(iris['sepal_width'], iris['species'], alpha=0.5)
print(result)

    Multiple Comparison of Means - Tukey HSD, FWER=0.50     
  group1     group2   meandiff p-adj   lower   upper  reject
------------------------------------------------------------
    setosa versicolor   -0.658    0.0 -0.7345 -0.5815   True
    setosa  virginica   -0.454    0.0 -0.5305 -0.3775   True
versicolor  virginica    0.204 0.0088  0.1275  0.2805   True
------------------------------------------------------------


#### 4-30. 이원분산분석(Two-way ANOVA)
* **독립변인(범주형)의 수가 2개일 때** 집단 간 종속변수(연속형)의 평균차이가 유의한지를 검증하는데 사용
* 두 독립변수간 상호작용효과가 있는지 확인할 수 있음
* 즉, 한 독립변수의 변화가 결과에 미치는 영향이 다른 독립변수의 수준에 따라 달라지는지를 확인할 수 있음.
* statsmodels 라이브러리 이용

**성별과 요일에 따른 팁 평균 차이가 있을까?**
* 독립변수: 성별, 요일
* 종속변수: tip
* Two-one ANOVA를 통해 통계적 유의성을 알아 볼 수 있다.
* 귀무가설: 성별(1)이나 요일(2)에 따른 팁의 평균차이가 없다.
* 대립가설: 성별(1)이나 요일(2)에 따른 팁의 평균창이가 있다.
* 귀무가설3: 성별과 요일 간의 상호작용 효과가 없다.
* 대립가설3: 성별과 요일 간의 상호작용 효과가 있다.

In [21]:
# [1] 데이터 가져오기
import pandas as pd
url = "https://raw.githubusercontent.com/Soyoung-Yoon/bigdata/main/tips.csv"
tips = pd.read_csv(url)
print(tips.tail(5))

     total_bill   tip     sex smoker   day    time  size
239       29.03  5.92    Male     No   Sat  Dinner     3
240       27.18  2.00  Female    Yes   Sat  Dinner     2
241       22.67  2.00    Male    Yes   Sat  Dinner     2
242       17.82  1.75    Male     No   Sat  Dinner     2
243       18.78  3.00  Female     No  Thur  Dinner     2


In [22]:
# [2] 성별별 팁의 평균, 요일별 팁의 평균
print(tips.groupby(['sex'])['tip'].mean())
print(tips.groupby(['day'])['tip'].mean())

sex
Female    2.833448
Male      3.089618
Name: tip, dtype: float64
day
Fri     2.734737
Sat     2.993103
Sun     3.255132
Thur    2.771452
Name: tip, dtype: float64


In [26]:
# [3] 이원분산분석
from statsmodels.api import OLS
from statsmodels.stats.anova import anova_lm

# 교호작용 불포함
formula = "tip ~ C(sex) + C(day)"
model = OLS.from_formula(formula, data=tips).fit()
result = anova_lm(model)
# print(result) # sex, day에 따른 tip의 평균차이가 없다.

# 교호작용 포함(독립변수끼리 서로 영향을 미치는지, 상호작용)
# formula2 = "tip ~ C(sex) + C(day) + C(sex):C(day)"
formula2 = "tip ~ C(sex) * C(day)"
model2 = OLS.from_formula(formula2, data=tips).fit()
result2 = anova_lm(model2)
print(result2)

                  df      sum_sq   mean_sq         F    PR(>F)
C(sex)           1.0    3.673534  3.673534  1.920989  0.167056
C(day)           3.0    7.446900  2.482300  1.298061  0.275785
C(sex):C(day)    3.0    2.785891  0.928630  0.485606  0.692600
Residual       236.0  451.306151  1.912314       NaN       NaN


**펭귄 데이터 셋**
* 펭귄의 몸무게(body_mass_g)에 성별(sex)과 종(species)에 따른 평균의 차이가 있는지 검정
* 성별과 종 간의 교호작용(상호작용)효과를 갖고 몸무게에 영향을 미치는지 확인

In [28]:
import pandas as pd
from statsmodels.api import OLS
from statsmodels.stats.anova import anova_lm

url = "https://raw.githubusercontent.com/Soyoung-Yoon/bigdata/main/penguins.csv"
df = pd.read_csv(url)
# print(df.head(3))

# 성별, 종별 몸무게의 평균 확인
print(df.groupby('sex')['body_mass_g'].mean())
print(df.groupby('species')['body_mass_g'].mean())

sex
Female    3862.272727
Male      4545.684524
Name: body_mass_g, dtype: float64
species
Adelie       3706.164384
Chinstrap    3733.088235
Gentoo       5092.436975
Name: body_mass_g, dtype: float64


In [31]:
# 교호작용 불포함
formula = 'body_mass_g ~ C(sex) + C(species)'
model = OLS.from_formula(formula, data=df).fit()
result = anova_lm(model)
print(result)

# 교호작용 포함(독립변수끼리 서로 영향을 미치는지, 상호작용
formula2 = 'body_mass_g ~ C(sex) * C(species)'
model2 = OLS.from_formula(formula2, data=df).fit()
result2 = anova_lm(model2)
# print(result2)


               df        sum_sq       mean_sq           F         PR(>F)
C(sex)        1.0  3.887890e+07  3.887890e+07  387.855463   1.366199e-57
C(species)    2.0  1.434016e+08  7.170079e+07  715.286340  1.618971e-120
Residual    329.0  3.297919e+07  1.002407e+05         NaN            NaN


In [33]:
# [2] 결정계수 구하기
SST = result['sum_sq'].sum()
SSE = result.loc['Residual', 'sum_sq']
SSR = SST - SSE
R2 = SSR / SST
print(round(R2, 4))

0.8468


In [34]:
# [3] mse 구하기
mse = result.loc['Residual', 'mean_sq']
print(round(mse, 4))

100240.684
